In [1]:
#requests is needed to connect to the API
import requests
import pandas as pd

In [27]:
def get_request(game, year='2018', season_type='02'):
    '''Input - takes 3 arguements game-integer, year-string, season_type-string. 
               game is an integer for the game number id, minus any prefixing 0's
               year is the first year number of a season i.e 2018 = 2018/2019 season. 2018 is the default
               season_type represents the type of season used. Default 2 represents the regular season
       Output - Returns a josn file from the API for the game requested
       Desc - This function takes input from the user to make a request to the API for a specified game
              It then jsonifies the request and returns that json file
    '''
    r = requests.get(url='http://statsapi.web.nhl.com/api/v1/game/'
                     +year+season_type+str(game).zfill(4)+'/feed/live')
    data = r.json()
    return data

def get_meta_stats(data):
    '''Input - Data from the NHL_API
       Output - returns the name and final score of the home and away team
       Desc - takes in thet data gotten from the api, and collects the data on the teams and score
       it then returns this information
    '''
    core = data['liveData']['boxscore']['teams']
    home_team = core['home']['team']['triCode']
    home_score = core['home']['teamStats']['teamSkaterStats']['goals']
    away_team = core['away']['team']['triCode']
    away_score = core['away']['teamStats']['teamSkaterStats']['goals']
    
    return home_team, home_score, away_team, away_score
    

def get_corsi(data, home_team, away_team):
    '''Input - Data from NHL_API, the home and away teams triCode id
       Output - Returns the number of corsi events for each team
       Description - This ffunction is what scrolls through the play by play data and counts 
       all the times there is a corsi event, which is either a shot, blocked shot, or missed shot
       It then returns this calue for both teams
    '''
    event_types = ['Shot','Blocked Shot','Missed Shot']
    home_event = 0
    away_event = 0    
    plays = data['liveData']['plays']['allPlays'] 
    for play in plays:
        if play['result']['event'] in event_types:
            if play['team']['triCode'] == home_team:
                home_event += 1
            if play['team']['triCode'] == away_team:
                away_event += 1
    
    return home_event, away_event

def make_a_list(game_id, home_team, home_score, away_team, away_score, home_corsi, away_corsi):
    '''Input - all the variables that have been collected. 
       Output - A single list of all the variables 
    '''
    row_list = []
    row_list.append(game_id)
    row_list.append(home_team)
    row_list.append(home_score)
    row_list.append(home_corsi)
    row_list.append(away_team)
    row_list.append(away_score)
    row_list.append(away_corsi)
    return row_list

In [ ]:
to_df_list = []
for i in range(1,1302):    
    data = get_request(game=i)
    home_team, home_score, away_team, away_score = get_meta_stats(data)
    home_corsi, away_corsi = get_corsi(data, home_team, away_team)
    to_df_list.append(make_a_list(i, home_team, home_score, away_team, away_score, home_corsi, away_corsi))

In [30]:
to_df_list = []
for i in range(1,1302):    
    data = get_request(game=i)
    home_team, home_score, away_team, away_score = get_meta_stats(data)
    home_corsi, away_corsi = get_corsi(data, home_team, away_team)
    to_df_list.append(make_a_list(i, home_team, home_score, away_team, away_score, home_corsi, away_corsi))
    

KeyError: 'liveData'

In [32]:
len(to_df_list)

1271